<a href="https://colab.research.google.com/github/prikmm/Breast_Cancer_Detection/blob/master/notebooks/breast-cancer-comprehensive-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import tarfile
import urllib

"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

# Importing Libraries:

## Libraries for EDA & Feature-Selection:

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
!pip install seaborn --upgrade
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
!pip install swiftviz
import swiftviz as sv
!pip install pandas_profiling
import pandas_profiling

# Feature Selection
!pip install ppscore
import ppscore as pps

Requirement already up-to-date: seaborn in /usr/local/lib/python3.6/dist-packages (0.11.1)
     |████████████████████████████████| 51kB 2.5MB/s 
  Created wheel for ppscore: filename=ppscore-1.2.0-py2.py3-none-any.whl size=13050 sha256=a05d761b6136ef910c8007f23957281e1674540b238be8cf26cc9c338badf575
  Stored in directory: /root/.cache/pip/wheels/50/cc/8f/d0da07e4ec4e7a4c4cf32aee9057e4d1c318de910543b4684e
Successfully built ppscore


In [3]:
sns.__version__

'0.11.1'

## Libraries for Preprocessing & Model-Selection:

In [4]:
# Library for pre-processing:
from sklearn.preprocessing import StandardScaler

# Library for Dimensionality-Reduction:
from sklearn.decomposition import PCA

# Libraries for modelling
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
!pip install catboost
from catboost import CatBoostClassifier


# Model Selection:
from sklearn.pipeline import Pipeline as Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.base import clone

# Libraries for model evaluaton 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_predict

# --CLASSIFICATION:
from sklearn import metrics

# Library for plotting confusion matrix
from mlxtend.plotting import plot_confusion_matrix

# Miscellanous libraries
from IPython.display import display

     |████████████████████████████████| 65.8MB 55kB/s 


## Loading Dataset:

In [5]:
bc_df = pd.read_csv('https://raw.githubusercontent.com/prikmm/Breast_Cancer_Detection/master/data/data.csv')
bc_df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [6]:
bc_df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [7]:
bc_df.drop(columns=['id', 'Unnamed: 32'], inplace=True)

The data we have consists of mean, standard deviation, and worst measure (average of top 3 maximum values). From this we can already judge that, features will have very strong correlation between them. But, Let's check for null values.

In [8]:
bc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

### No Null-values & every column has the correct data-type. All good!

In [9]:
report = bc_df.describe().T
report

,count,mean,std,min,25%,50%,75%,max
radius_mean,569.0,14.127292,3.524049,6.981000,11.700000,13.370000,15.780000,28.11000
texture_mean,569.0,19.289649,4.301036,9.710000,16.170000,18.840000,21.800000,39.28000
perimeter_mean,569.0,91.969033,24.298981,43.790000,75.170000,86.240000,104.100000,188.50000
area_mean,569.0,654.889104,351.914129,143.500000,420.300000,551.100000,782.700000,2501.00000
smoothness_mean,569.0,0.096360,0.014064,0.052630,0.086370,0.095870,0.105300,0.16340
compactness_mean,569.0,0.104341,0.052813,0.019380,0.064920,0.092630,0.130400,0.34540
concavity_mean,569.0,0.088799,0.079720,0.000000,0.029560,0.061540,0.130700,0.42680
concave points_mean,569.0,0.048919,0.038803,0.000000,0.020310,0.033500,0.074000,0.20120
symmetry_mean,569.0,0.181162,0.027414,0.106000,0.161900,0.179200,0.195700,0.30400
fractal_dimension_mean,569.0,0.062798,0.007060,0.049960,0.057700,0.061540,0.066120,0.09744


In [10]:
features = np.array(report.index)
target = ['diagnosis']
features

array(['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se',
       'smoothness_se', 'compactness_se', 'concavity_se',
       'concave points_se', 'symmetry_se', 'fractal_dimension_se',
       'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst',
       'smoothness_worst', 'compactness_worst', 'concavity_worst',
       'concave points_worst', 'symmetry_worst',
       'fractal_dimension_worst'], dtype=object)

# Data Exploration:

#### Setting default custom palette:

In [11]:
cust_palette = sv.CustomPalette()
cust_palette.display_palette('Dark2')
cust_palette.set_default_custom_palette('Dark2')

In [12]:
print(bc_df["diagnosis"].value_counts())
sns.countplot(x=bc_df["diagnosis"]);

B    357
M    212
Name: diagnosis, dtype: int64


In [13]:
axis = sv.Plotter.row_col_merge(rows=2, cols=5)

### Distributions of Means:

In [14]:
cust_palette.set_default_custom_palette('dark')
axis_var_list = list(zip(axis, features[:10]))

fig, axs = plt.subplots(figsize=(30, 10), nrows=2, ncols=5)

for axis_tup, var in axis_var_list:
    row = axis_tup[0]
    col = axis_tup[1]
    sns.histplot(data=bc_df, x=var, ax=axs[row, col], hue="diagnosis")

### Distribution of Standard Errors:

In [15]:
cust_palette.set_default_custom_palette('Set1')
axis_var_list = list(zip(axis, features[10:20]))

fig, axs = plt.subplots(figsize=(30, 10), nrows=2, ncols=5)

for axis_tup, var in axis_var_list:
    row = axis_tup[0]
    col = axis_tup[1]
    sns.histplot(data=bc_df, x=var, ax=axs[row, col], hue="diagnosis")

### Distributions of Worst:

In [16]:
cust_palette.set_default_custom_palette('Set2')
axis_var_list = list(zip(axis, features[20:]))

fig, axs = plt.subplots(figsize=(30, 10), nrows=2, ncols=5)

for axis_tup, var in axis_var_list:
    row = axis_tup[0]
    col = axis_tup[1]
    sns.histplot(data=bc_df, x=var, ax=axs[row, col], hue="diagnosis")

## Outlier Analysis:

### Remember: 
Larger the distance between the two medians('M' & 'B'), more useful the feature.<br>
This is because larger the distance between the medians<br>
means larger the distance between clusters, this will make the clusters to be more distinguishable.

In [17]:
outlier_handler = sv.OutlierAnalysis(bc_df)
scaled_bc_df = outlier_handler.data_scaler(list(features))

### Outlier Report For Means:

In [18]:
cust_palette.set_default_custom_palette('ocean_r')
data_mean = pd.melt(pd.concat([scaled_bc_df[features[:10]], bc_df[target]], axis=1), 
                    id_vars="diagnosis",
                    var_name="features",
                    value_name='value')

fig, axs = plt.subplots(figsize=(30, 10))

display(sns.violinplot(data=data_mean, x="features", y="value", hue="diagnosis", inner="quart",
                       split=True));

mean_outlier_report = outlier_handler.OutlierReport(list(features[:10]))
mean_outlier_report

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
median,13.370,18.840,86.240,551.1,0.095870,0.09263,0.06154,0.033500,0.1792,0.06154
min,6.981,9.710,43.790,143.5,0.052630,0.01938,0.00000,0.000000,0.1060,0.04996
max,28.110,39.280,188.500,2501.0,0.163400,0.34540,0.42680,0.201200,0.3040,0.09744
25%,11.700,16.170,75.170,420.3,0.086370,0.06492,0.02956,0.020310,0.1619,0.05770
75%,15.780,21.800,104.100,782.7,0.105300,0.13040,0.13070,0.074000,0.1957,0.06612
IQR,4.080,5.630,28.930,362.4,0.018930,0.06548,0.10114,0.053690,0.0338,0.00842
Left Whisker Limit,5.580,7.725,31.775,-123.3,0.057975,-0.03330,-0.12215,-0.060225,0.1112,0.04507
Right Whisker Limit,21.900,30.245,147.495,1326.3,0.133695,0.22862,0.28241,0.154535,0.2464,0.07875
No.of Left Outliers,0.000,0.000,0.000,0.0,1.000000,0.00000,0.00000,0.000000,1.0000,0.00000
No.of Right Outliers,14.000,7.000,13.000,25.0,5.000000,16.00000,18.00000,10.000000,14.0000,15.00000


The table shown is for the entire dataset, and the plots are shown to visualize w.r.t "diagnosis".

### Outlier Report for Standard Errors:

In [19]:
data_se = pd.melt(pd.concat([scaled_bc_df[features[10:20]], bc_df[target]], axis=1), 
                    id_vars="diagnosis",
                    var_name="features",
                    value_name='value')

fig, axs = plt.subplots(figsize=(30, 10))

display(sns.violinplot(data=data_se, x="features", y="value", hue="diagnosis", inner="quart",
                       split=True, palette=sns.color_palette('Dark2')[3:6:2]));

se_outlier_report = outlier_handler.OutlierReport(list(features[10:20]))
se_outlier_report

,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se
median,0.32420,1.10800,2.2870,24.530,0.006380,0.020450,0.02589,0.010930,0.018730,0.003187
min,0.11150,0.36020,0.7570,6.802,0.001713,0.002252,0.00000,0.000000,0.007882,0.000895
max,2.87300,4.88500,21.9800,542.200,0.031130,0.135400,0.39600,0.052790,0.078950,0.029840
25%,0.23240,0.83390,1.6060,17.850,0.005169,0.013080,0.01509,0.007638,0.015160,0.002248
75%,0.47890,1.47400,3.3570,45.190,0.008146,0.032450,0.04205,0.014710,0.023480,0.004558
IQR,0.24650,0.64010,1.7510,27.340,0.002977,0.019370,0.02696,0.007072,0.008320,0.002310
Left Whisker Limit,-0.13735,-0.12625,-1.0205,-23.160,0.000704,-0.015975,-0.02535,-0.002970,0.002680,-0.001217
Right Whisker Limit,0.84865,2.43415,5.9835,86.200,0.012611,0.061505,0.08249,0.025318,0.035960,0.008023
No.of Left Outliers,0.00000,0.00000,0.0000,0.000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
No.of Right Outliers,38.00000,20.00000,38.0000,65.000,30.000000,28.000000,22.00000,19.000000,27.000000,28.000000


### Outlier Report for Worst:

In [20]:
data_worst = pd.melt(pd.concat([scaled_bc_df[features[20:30]], bc_df[target]], axis=1), 
                    id_vars="diagnosis",
                    var_name="features",
                    value_name='value')

fig, axs = plt.subplots(figsize=(30, 10))

display(sns.violinplot(data=data_worst, x="features", y="value", hue="diagnosis", inner="quart",
                       split=True, palette=sns.color_palette('viridis')[3:6:2]));

worst_outlier_report = outlier_handler.OutlierReport(list(features[20:30]))
worst_outlier_report

,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
median,14.97,25.41,97.660,686.50,0.13130,0.21190,0.2267,0.099930,0.28220,0.08004
min,7.93,12.02,50.410,185.20,0.07117,0.02729,0.0000,0.000000,0.15650,0.05504
max,36.04,49.54,251.200,4254.00,0.22260,1.05800,1.2520,0.291000,0.66380,0.20750
25%,13.01,21.08,84.110,515.30,0.11660,0.14720,0.1145,0.064930,0.25040,0.07146
75%,18.79,29.72,125.400,1084.00,0.14600,0.33910,0.3829,0.161400,0.31790,0.09208
IQR,5.78,8.64,41.290,568.70,0.02940,0.19190,0.2684,0.096470,0.06750,0.02062
Left Whisker Limit,4.34,8.12,22.175,-337.75,0.07250,-0.14065,-0.2881,-0.079775,0.14915,0.04053
Right Whisker Limit,27.46,42.68,187.335,1937.05,0.19010,0.62695,0.7855,0.306105,0.41915,0.12301
No.of Left Outliers,0.00,0.00,0.000,0.00,1.00000,0.00000,0.0000,0.000000,0.00000,0.00000
No.of Right Outliers,17.00,5.00,15.000,35.00,6.00000,16.00000,12.0000,0.000000,23.00000,24.00000


## Random Visualizations for Feature Selections:

We will use this visualizations for determining whether to use pearson's correlation or spearman's correlation

In [21]:
cust_palette.set_default_custom_palette(["#f7a400", "#3a9efd", "#3e4491", "#292a73", "#1a1b4b"])
plot_maker = sv.Plotter()
plot_maker.plotter(bc_df, 'radius_mean', ['texture_mean', 'perimeter_mean', 'smoothness_mean', 'concavity_mean', 'fractal_dimension_mean', 'area_mean'],
                   ['diagnosis'], 30, 7)

In [22]:
plot_maker.plotter(bc_df, 'radius_se', ['texture_se', 'perimeter_se', 'smoothness_se', 'concavity_se', 'fractal_dimension_se', 'area_se'], ['diagnosis'], 30, 5)

In [23]:
plot_maker.plotter(bc_df, 'radius_worst', ['texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'fractal_dimension_worst'], ['diagnosis'], 30, 5)

In [24]:
plot_maker.plotter(bc_df, 'radius_mean', ['radius_se', 'radius_worst', 'area_se', 'area_worst', 'perimeter_se', 'perimeter_worst'], ['diagnosis'], 30, 5)

## Heatmap: (Using Pearson's Corr)

In [25]:
corr_matrix = bc_df.corr()

fig, axs = plt.subplots(figsize=(20, 15))
sns.heatmap(corr_matrix, annot=True)

Let's filter out the values using a threshold, it will be a lot easier for us to find strong correlations:

In [26]:
threshold = 0.8
cond_corr_matrix = corr_matrix[(corr_matrix > threshold) | (corr_matrix < -threshold)]

fig, axs = plt.subplots(figsize=(20, 10))
sns.heatmap(cond_corr_matrix, annot=True)

We can see many strong correlations from the matrix, they are:

* radius_mean, perimeter_mean, area_mean, radius_worst, perimeter_worst, area_worst
* radius_se, perimeter_se, area_se
* compactness_mean, concavity_mean, concave points_mean
* texture_mean, texture_worst
<br>

& many more, but, majority of the features from above are going to be removed as they have strong correlation with each other.<br>
Hence, we can derive one feature from another, so, it is of no use to keep all the strongly correlated features.

We will only keep 4 features from the above 4 points (1 for each) randomly. We will keep:
* area_mean (It has got more correlations than radius_mean)
* area_se
* concave points_mean
* texture_mean

In [27]:
drop_features = ['radius_mean', 'perimeter_mean', 'radius_worst', 'perimeter_worst',
                 'area_worst', 'radius_se', 'perimeter_se', 'compactness_mean', 'concave points_worst', 'smoothness_worst',
                 'concavity_mean', 'compactness_worst', 'texture_worst']

modified_bc_df = bc_df.drop(columns=drop_features)
modified_corr_matrix = modified_bc_df.corr()

fig, axs = plt.subplots(figsize=(20, 10))
sns.heatmap(modified_corr_matrix, annot=True)

# Model-Selection:

In [28]:
modified_bc_df.columns

Index(['diagnosis', 'texture_mean', 'area_mean', 'smoothness_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'texture_se', 'area_se', 'smoothness_se', 'compactness_se',
       'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'concavity_worst', 'symmetry_worst',
       'fractal_dimension_worst'],
      dtype='object')

In [29]:
X = modified_bc_df.drop(columns='diagnosis')
y = modified_bc_df.diagnosis

X_train, X_test_cv, y_train, y_test_cv = train_test_split(X, y, test_size=0.4)
X_val, X_test, y_val, y_test = train_test_split(X_test_cv, y_test_cv, test_size=0.5)

In [30]:
for name, x, y in (('Train', X_train, y_train), ('Validation', X_val, y_val), ('Test', X_test, y_test)):
    print(f'{name}:')
    print(f'Data: {x.shape}')
    print(f'Target: {y.shape}')
    print('')

Train:
Data: (341, 17)
Target: (341,)

Validation:
Data: (114, 17)
Target: (114,)

Test:
Data: (114, 17)
Target: (114,)



## PCA:

In [31]:
X_train_scaled = StandardScaler().fit_transform(X_train)
pca = PCA()
pca.fit(X_train_scaled)
cumsum = np.cumsum(pca.explained_variance_ratio_)
print(cumsum)

plt.figure(figsize=(10,7))
plt.plot(cumsum, linewidth=3)
plt.axis([0, 30, 0, 1])
plt.xlabel("n_components")
plt.ylabel("Explained Variance")
plt.grid(True)

[0.37704664 0.53639119 0.65884957 0.7456615  0.81059278 0.86692918
 0.89634613 0.92231859 0.94127769 0.9566284  0.96895773 0.97747627
 0.98537165 0.99235148 0.99572331 0.9983937  1.        ]


We can see, that for to get explained variance >= 95%, we need n_componenets >= 8

## Main Pipeline:

In [32]:
main_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('model', None),
])

In [56]:
rnd_state = 4

params_grid = [
    {
        "pca__n_components": [8, 9, 10, 11, 12],
        "model": [XGBClassifier()],
        "model__n_estimators": [100, 500, 1000],
        "model__learning_rate": [0.005, 0.01, 0.05, 0.1],
        "model__max_depth": [3, 4, 5, 6, 10],
        "model__gamma": [0, 1, 5],
        "model__random_state": [rnd_state],
    },
    {
        "pca__n_components": [8, 9, 10, 11, 12],
        "model": [GradientBoostingClassifier()],
        "model__n_estimators": [100, 1000],
        "model__learning_rate": [0.001, 0.05, 0.1, 0.5],
        "model__random_state": [rnd_state],
    },
    {
        "pca__n_components": [8, 9, 10, 11, 12],
        "model": [RandomForestClassifier()],
        "model__n_estimators": [100, 1000],
        "model__max_depth": [3, 4, 5, 6, 10, 15, 20],
        "model__random_state": [rnd_state],
    },
    {
        "pca__n_components": [8, 9, 10, 11, 12],
        "model": [LGBMClassifier()],
        "model__n_estimators": [100, 500, 1000],
        "model__learning_rate": [0.005, 0.01, 0.05, 0.1],
        "model__max_depth": [3, 4, 5, 6, 10, -1],
        "model__random_state": [rnd_state],
    },
    {
        "pca__n_components": [8, 9, 10, 11, 12],
        "model": [CatBoostClassifier()],
        "model__n_estimators": [100, 500, 1000],
        "model__learning_rate": [0.005, 0.01, 0.05, 0.1],
        "model__max_depth": [3, 4, 5, 6, 10],
        "model__random_state": [rnd_state],
    },
]

In [57]:
main_grid = GridSearchCV(main_pipeline, params_grid, cv=3, verbose=2, scoring="accuracy")

In [ ]:
main_grid.fit(X_train, y_train)

In [59]:
main_grid.best_estimator_

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('pca',
                 PCA(copy=True, iterated_power='auto', n_components=9,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x7f70194b6160>)],
         verbose=False)

In [63]:
y_pred = main_grid.best_estimator_.predict(X_val)
print(metrics.classification_report(y_val, y_pred))
metrics.plot_roc_curve(main_grid.best_estimator_, X_val, y_val);

              precision    recall  f1-score   support

           B       0.93      0.94      0.93        66
           M       0.91      0.90      0.91        48

    accuracy                           0.92       114
   macro avg       0.92      0.92      0.92       114
weighted avg       0.92      0.92      0.92       114



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_plot/roc_curve.py:87: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



In [64]:
conf_matrix = metrics.confusion_matrix(y_val, y_pred)
plot_confusion_matrix(conf_mat=conf_matrix);

/usr/local/lib/python3.6/dist-packages/mlxtend/plotting/plot_confusion_matrix.py:61: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



## Trying to increase model accuracy using Soft Vote Ensembling:

In [65]:
best_params = main_grid.best_params_
best_params

{'model': <catboost.core.CatBoostClassifier at 0x7f7019881f28>,
 'model__learning_rate': 0.005,
 'model__max_depth': 3,
 'model__n_estimators': 1000,
 'model__random_state': 4,
 'pca__n_components': 9}

In [ ]:
best_pipeline = clone(main_grid.best_estimator_)

best_pipeline.fit(X_train, y_train)

In [75]:
y_pred = best_pipeline.predict(X_val)
print(metrics.classification_report(y_val, y_pred))
metrics.plot_roc_curve(best_pipeline, X_val, y_val);

              precision    recall  f1-score   support

           B       0.94      0.95      0.95        66
           M       0.94      0.92      0.93        48

    accuracy                           0.94       114
   macro avg       0.94      0.94      0.94       114
weighted avg       0.94      0.94      0.94       114



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_plot/roc_curve.py:87: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



In [76]:
conf_matrix = metrics.confusion_matrix(y_val, y_pred)
plot_confusion_matrix(conf_mat=conf_matrix);

/usr/local/lib/python3.6/dist-packages/mlxtend/plotting/plot_confusion_matrix.py:61: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



Creating second CatBoostClassifier Pipeline for VotingEnsemble:

In [77]:
second_pipeline = Pipeline([
                ("pca", PCA()),
                ("cat", CatBoostClassifier()),
])

In [78]:
second_param_grid = [
        {
            "pca__n_components": [8, 9, 10, 11, 12],
            "cat__n_estimators": [100, 500, 1000],
            "cat__learning_rate": [0.005, 0.01, 0.05, 0.1],
            "cat__max_depth": [3, 4, 5, 6, 10],
            "cat__random_state": [rnd_state],
        }
]

In [ ]:
main_grid_2 = GridSearchCV(second_pipeline, second_param_grid, cv=3, verbose=2, scoring='accuracy')
main_grid_2.fit(X_train, y_train)

In [120]:
main_grid_2.best_estimator_

Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=8,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('cat',
                 <catboost.core.CatBoostClassifier object at 0x7f7019792a90>)],
         verbose=False)

In [123]:
best_params_2 = main_grid_2.best_params_
best_params_2

{'cat__learning_rate': 0.05,
 'cat__max_depth': 3,
 'cat__n_estimators': 1000,
 'cat__random_state': 4,
 'pca__n_components': 8}

In [82]:
y_pred = main_grid_2.best_estimator_.predict(X_val)
print(metrics.classification_report(y_val, y_pred))
metrics.plot_roc_curve(main_grid_2.best_estimator_, X_val, y_val);

              precision    recall  f1-score   support

           B       0.94      0.98      0.96        66
           M       0.98      0.92      0.95        48

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_plot/roc_curve.py:87: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



Let's combine the two estimators using Soft Voting Ensemble:

In [83]:
best_pipeline_2 = clone(main_grid_2.best_estimator_)

In [84]:
softVoter = VotingClassifier(estimators=[
        ('cat_1', best_pipeline), ('cat_2', best_pipeline_2)], voting="soft")

In [ ]:
softVoter.fit(X_train, y_train)

In [86]:
y_pred = softVoter.predict(X_val)
print(metrics.classification_report(y_val, y_pred))
metrics.plot_roc_curve(softVoter, X_val, y_val);

              precision    recall  f1-score   support

           B       0.94      0.97      0.96        66
           M       0.96      0.92      0.94        48

    accuracy                           0.95       114
   macro avg       0.95      0.94      0.95       114
weighted avg       0.95      0.95      0.95       114



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_plot/roc_curve.py:87: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



#### We can clearly see that out of all, the classifiers - first CatBoost, second CatBoost & Soft Voting Ensemble, The one with best result is second CatBoost. So, we will use joblib to save that classifier:

In [87]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [117]:
import joblib
import pickle

In [134]:
pca_final = clone(best_pipeline_2.steps[0][1])
cat_clf = clone(best_pipeline_2.steps[1][1])

In [ ]:
new_X_train = pca_final.fit_transform(X_train)
cat_clf.fit(new_X_train, y_train)

In [136]:
print("For test:")
print(metrics.classification_report(y_test, y_pred_test))
metrics.plot_roc_curve(cat_clf, new_X_test, y_test);

For test:
              precision    recall  f1-score   support

           B       0.98      0.99      0.98        81
           M       0.97      0.94      0.95        33

    accuracy                           0.97       114
   macro avg       0.97      0.96      0.97       114
weighted avg       0.97      0.97      0.97       114



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_plot/roc_curve.py:87: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



#### Great, we got a good result with test set. So, we will choose this model:

In [ ]:
for estimator in (pca_final, cat_clf):
    joblib.dump(estimator, f'/content/drive/MyDrive/{str(estimator.__class__).replace(".", "_")[8:-2]}.pkl')